# LangChain Use Cases

From the tutorial [gkamradt/langchain-tutorials](https://github.com/gkamradt/langchain-tutorials/blob/main/LangChain%20Cookbook%20Part%202%20-%20Use%20Cases.ipynb)

### toc
- [Summarization](#Summarization)
- [O&A over documents](#Resources)
- Extraction
- Evaluation
- Querying tabular data
- Code understanding
- Interacting with APIs
- Chatbots
- Agents


There are also some wonderful projects & use cases: https://github.com/gkamradt/langchain-tutorials




### Summarization

Summarize from a bunch of information with the help from LLMs, like texts (possibly be books, articles, documents in any fields, table, financial reports, user manuals, podcasts, online posts, etc.)

#### Summaries of short text

Just simply prompt with instructions:

In [ ]:
from langchain.llms import OpenAI
